# Leverage calculations from market data

A simple notebook showing leverage calculations based on market data obtined via the API.

Resources:
- [Testnet](https://console.fairground.wtf/)
- [API docs](https://docs.fairground.vega.xyz/docs/apis/)

## [gRPC](https://docs.fairground.vega.xyz/docs/apis/grpc/)

Uncomment the cell below to install the [Vega-API-client](https://pypi.org/project/Vega-API-client/) for Python if you've not done it yet.

In [3]:
#pip install --upgrade Vega-API-client;

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 119 kB 4.6 MB/s 
     |████████████████████████████████| 3.9 MB 5.0 MB/s 
     |████████████████████████████████| 198 kB 8.5 MB/s 
     |████████████████████████████████| 1.0 MB 6.5 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import vegaapiclient as vac
import math
node_url_grpc="n06.testnet.vega.xyz:3002"
data_client = vac.VegaTradingDataClient(node_url_grpc)

def round_sd(number):
    significant_digits = 4
    return round(number, significant_digits - math.floor(math.log10(abs(number))) - 1)

### Specify public key for the party

Specify the your public key in the cell below. It can be found in the [wallet side panel](https://console.fairground.wtf/wallet/) in the Console.

![](./resources/pubkey.png)

In [5]:
pubkey="9f6140abd99e18848e89f8bd9fdad34d9b04409f2c998688205de33016522fde"

In [15]:
party_positions = data_client.PositionsByParty(vac.api.trading.PositionsByPartyRequest(party_id=pubkey))
if len(party_positions.positions) == 0:
    print("party with the public key specified has no open positions")
    exit()

party_accounts = data_client.PartyAccounts(vac.api.trading.PartyAccountsRequest(party_id=pubkey))

for pos in party_positions.positions:
    market_name=data_client.MarketByID(vac.api.trading.MarketByIDRequest(market_id=pos.market_id)).market.tradable_instrument.instrument.name
    market_data=data_client.MarketDataByID(vac.api.trading.MarketDataByIDRequest(market_id=pos.market_id)).market_data
    margin_acc = next(x for x in party_accounts.accounts if x.market_id == pos.market_id )
    gen_acc = next(x for x in party_accounts.accounts if x.type == vac.vega.ACCOUNT_TYPE_GENERAL and x.asset==margin_acc.asset)
    
    notional=pos.open_volume*market_data.mark_price
    current_leverage=round_sd(notional/(gen_acc.balance+margin_acc.balance))
    capital_utilisation=round_sd((margin_acc.balance/gen_acc.balance)*100)

    margin_levels=data_client.MarginLevels(vac.api.trading.MarginLevelsRequest(party_id=pubkey, market_id=pos.market_id)).margin_levels

    print("\tmarket: {name}".format(name=market_name)) 
    print("\t\teffective position leverage: ~{lev}x".format(lev=current_leverage))
    print("\t\tleverage per margin level:")
    print("\t\t\t{:<20} ~{}x".format("maintenance:",round_sd(notional/margin_levels[0].maintenance_margin)))
    print("\t\t\t{:<20} ~{}x".format("search",round_sd(notional/margin_levels[0].search_level)))
    print("\t\t\t{:<20} ~{}x".format("initial",round_sd(notional/margin_levels[0].initial_margin)))
    print("\t\t\t{:<20} ~{}x".format("release",round_sd(notional/margin_levels[0].collateral_release_level)))
    print("\t\tcapital utilisation: ~{lev}%".format(lev=capital_utilisation))  
    print("\t\t\t{:<20} ~{}%".format("maintenance:",round_sd(margin_levels[0].maintenance_margin)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("search",round_sd(margin_levels[0].search_level)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("initial",round_sd(margin_levels[0].initial_margin)/notional*100))
    print("\t\t\t{:<20} ~{}%".format("release",round_sd(margin_levels[0].collateral_release_level)/notional*100))


	market: BTCUSD Monthly (30 Jun 2021)
		effective position leverage: ~0.7768x
		leverage per margin level:
			maintenance:         ~46.75x
			search               ~45.61x
			initial              ~44.53x
			release              ~19.0x
		capital utilisation: ~2.198%
			maintenance:         ~2.1383335907812353%
			search               ~2.192370901739606%
			initial              ~2.2464082126979767%
			release              ~5.264778010515533%
	market: AAVEDAI Monthly (30 Jun 2021)
		effective position leverage: ~0.003131x
		leverage per margin level:
			maintenance:         ~70.7x
			search               ~68.97x
			initial              ~67.33x
			release              ~64.27x
		capital utilisation: ~0.00465%
			maintenance:         ~1.4145696498078846%
			search               ~1.4498033956979335%
			initial              ~1.4853633799758532%
			release              ~1.5558308717559506%


## [GraphQL](https://docs.fairground.vega.xyz/docs/apis/graphql/)

All the data required for above calculations can also be obtained via the GraphQL API. Try going to the [GraphQL playground](https://lb.testnet.vega.xyz/playground) for the testnet and entering the query below (remember to modify the public key to yours) 

```bash
query($pubKey: ID = "e23f51fa4236ac4c843b56fed310db713055dae5d5ce71b976465295206b11bf") {        
  party(id:$pubKey){
    positions {
      market {
        name,
      	data {
          markPrice
        }}
      openVolume,
      margins {
        maintenanceLevel
       	searchLevel,
        initialLevel,
        collateralReleaseLevel,
      }
    },
    accounts {
      type,
      balance,
      asset {
        name
      }, 
      market {
        name
      }
    }
  }
}
```